# Detecting EMA cross traps by using NEAT

### Import Library

In [12]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import neat

### Load Price Data

In [14]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [15]:
data = dataset.copy()

In [16]:
data = data[data.index > '2020-11-01 00:00:00']

In [17]:
data["ema_fast"] = ta.ema(data["Close"], length=20)
data["ema_low"] = ta.ema(data["Close"], length=250)
data["ema_cross"] = ((data["ema_fast"] > data["ema_low"]) & (data["ema_fast"].shift(1) <= data["ema_low"].shift(1)) | (data["ema_fast"] < data["ema_low"]) & (data["ema_fast"].shift(1) >= data["ema_low"].shift(1)))

## Calculate some common features

In [18]:
data["ATR"] = ta.atr(data["High"], data["Low"], data["Close"], length=14)  # Volatility
data["RSI"] = ta.rsi(data["Close"], length=14)  # Momentum indicator

## TRAP labeling

In [19]:
def is_trap(r):
    trap = ''
    if r['ema_cross'] == True:
        if r['ema_fast'] > r['ema_low']:
            # Cross up
            if r['min_low_1dlater'] < r['Close'] - 3.5:
                trap = 1
            else:
                trap = 0
        else:
            # Cross down
            if r['max_high_1dlater'] > r['Close'] + 3.5:
                trap = 1
            else:
                trap = 0
    return trap

In [20]:
data['max_high_1dlater'] = data['High'].shift(-51).rolling(51).max()
data['min_low_1dlater'] = data['Low'].shift(-51).rolling(51).min()
data['trap'] = data.apply(lambda r: is_trap(r), axis=1)

In [21]:
# cross_data = data[data.ema_cross == True]
# len(cross_data[cross_data.trap == 0]) / len(cross_data['trap'])

## Features

In [22]:
data['diff_ema'] = data['Close'] - data['ema_fast']
data['diff_ema_s1'] = data['diff_ema'].shift(1)
data['diff_ema_s2'] = data['diff_ema'].shift(2)
data['diff_ema_s3'] = data['diff_ema'].shift(3)
data['diff_ema_s4'] = data['diff_ema'].shift(4)
data['diff_ema_s5'] = data['diff_ema'].shift(5)
data['diff_ema_s6'] = data['diff_ema'].shift(6)
data['diff_ema_s7'] = data['diff_ema'].shift(7)
data['diff_ema_s8'] = data['diff_ema'].shift(8)
cross_up_data = data[(data.ema_cross == True) & (data.ema_fast > data.ema_low)]
cross_up_data.dropna(inplace=True)

In [23]:
len(cross_up_data)

181

In [24]:
X = cross_up_data[['diff_ema', 'diff_ema_s1', 'diff_ema_s2', 'diff_ema_s3', 'diff_ema_s4', 'diff_ema_s5', 'diff_ema_s6', 'diff_ema_s7', 'diff_ema_s8', "trap"]]

# Train-Test Split
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)

In [25]:
len(X_train)

135

In [26]:
len(X_test)

46

In [27]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for move_index, row in X_train.iterrows():
            inputs = [row['diff_ema'], row['diff_ema_s1'], row['diff_ema_s2'], row['diff_ema_s3'], row['diff_ema_s4'], row["diff_ema_s5"], row["diff_ema_s6"], row["diff_ema_s7"], row["diff_ema_s8"]]
            expected_output = row['trap']
            output = net.activate(inputs)
            genome.fitness -= (output[0] - expected_output) ** 2


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    # p.add_reporter(neat.StdOutReporter(True))
    # stats = neat.StatisticsReporter()
    # p.add_reporter(stats)

    # Run for up to 100 generations.
    winner = p.run(eval_genomes, 100)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    return neat.nn.FeedForwardNetwork.create(winner, config)

In [28]:
%%time
config_path = os.path.join(current_dir, 'style-mix-1.cfg')
best_brain = run(config_path)


Best genome:
Key: 14038
Fitness: -25.799271944390664
Nodes:
	0 DefaultNodeGene(key=0, bias=0.11949945242186549, response=1.0, activation=sigmoid, aggregation=sum)
	1754 DefaultNodeGene(key=1754, bias=-0.8978117199083647, response=1.0, activation=sigmoid, aggregation=sum)
	1831 DefaultNodeGene(key=1831, bias=-0.6467929946651705, response=1.0, activation=sigmoid, aggregation=sum)
	1974 DefaultNodeGene(key=1974, bias=1.9614069725463672, response=1.0, activation=sigmoid, aggregation=sum)
	2005 DefaultNodeGene(key=2005, bias=-1.8172030343573367, response=1.0, activation=sigmoid, aggregation=sum)
	2282 DefaultNodeGene(key=2282, bias=-1.7796437147318833, response=1.0, activation=sigmoid, aggregation=sum)
	2664 DefaultNodeGene(key=2664, bias=-2.4562654642888444, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-9, 1974), weight=-3.116061096229862, enabled=True)
	DefaultConnectionGene(key=(-9, 2282), weight=-0.35046993945627947, enabled=True)
	Default

In [29]:
best_brain

In [30]:
# Show output of the most fit genome against training data.
outputs = []
for i, row in X_test.iterrows():
    inputs = [row['diff_ema'], row['diff_ema_s1'], row['diff_ema_s2'], row['diff_ema_s3'], row['diff_ema_s4'], row["diff_ema_s5"], row["diff_ema_s6"], row["diff_ema_s7"], row["diff_ema_s8"]]
    expected_output = row['trap']
    output = best_brain.activate(inputs)
    outputs.append(round(output[0]))
    # print("input {!r}, expected output {!r}, got {!r}".format(inputs, expected_output, output))

In [31]:
expected_outputs = X_test['trap'].to_list()
# Evaluate Performance
print("Accuracy:", accuracy_score(expected_outputs, outputs))

Accuracy: 0.7391304347826086
